In [1]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('r3.jpg')
original = image.copy()

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Thresholding to detect zones of inhibition
_, threshold = cv2.threshold(blurred, 170, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('original', threshold)
cv2.imwrite('threshold.jpg', threshold)
# Morphological operations to refine the mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
morph = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, kernel, iterations=2)

# Distance transform for better separation of overlapping zones
dist_transform = cv2.distanceTransform(morph, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Sure background by dilation
sure_bg = cv2.dilate(morph, kernel, iterations=3)

# Subtract unknown regions
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Markers for watershed
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Apply the watershed algorithm
markers = cv2.watershed(image, markers)
image[markers == -1] = [0, 0, 255]  # Mark boundaries in red

# Detect antibiotic discs using HSV
hsv = cv2.cvtColor(original, cv2.COLOR_BGR2HSV)
lower_yellow = np.array([20, 100, 100])  # Adjust for yellow range
upper_yellow = np.array([30, 255, 255])
mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

# Find contours of antibiotic discs
pill_contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Annotate zones of inhibition
for label in range(2, np.max(markers) + 1):  # Skip background and unknown
    zone_mask = (markers == label).astype(np.uint8) * 255
    cnts, _ = cv2.findContours(zone_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in cnts:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center = (int(x), int(y))
        radius = int(radius)
        if radius > 10:
            cv2.circle(original, center, radius, (0, 255, 0), 2)
            cv2.putText(original, f"Zone {label-1}", (center[0] - 20, center[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

# Annotate discs
for pill in pill_contours:
    (px, py), pill_radius = cv2.minEnclosingCircle(pill)
    pill_center = (int(px), int(py))
    pill_radius = int(pill_radius)
    cv2.circle(original, pill_center, pill_radius, (255, 0, 0), 2)
    cv2.putText(original, "Disc", (pill_center[0] - 20, pill_center[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

# Save and display the result
cv2.imwrite('result_advanced.jpg', original)
cv2.imshow("Result", original)
cv2.waitKey(0)
cv2.destroyAllWindows()
